# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/excertos_filtrados500.csv', sep = '|')
df.head()

,COD,DESCR_AREA,filtrado
0,1400,Responsabilidade,voto cuidar auto tomada conta especial instaur...
1,1700,Finanças Públicas,voto cuidar auto solicitação congresso naciona...
2,5700,Responsabilidade,relatório tratar embargo declaração opor exemp...
3,284,Direito Processual,voto relação outro processo judiciais tratar r...
4,298,Pessoal,voto relativo ato envolver senhor caber rememo...


In [2]:
df.shape

(13285, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['DESCR_AREA']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [4]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 30000
limite_texto = 500
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado'].astype(str))

sequences = tokenizer.texts_to_sequences(df['filtrado'].astype(str))

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 22972 unique tokens.


In [5]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 500)


In [6]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('../externos/model.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 22972
Encontrados no modelo: 19689 = 85.7086888385861


# Treinamento

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1126 00:02:12.507017 140615897368384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1126 00:02:12.642499 140615897368384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1126 00:02:12.644009 140615897368384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1126 00:02:12.651661 140615897368384 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 2,574,046
Trainable params: 276,746
Non-trainable params: 2,297,300
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 131s 12ms/step - loss: 1.3050 - categorical_accuracy: 0.5769 - val_loss: 1.2935 - val_categorical_accuracy: 0.5390
Epoch 2/20
10628/10628 [==============================] - 132s 12ms/step - loss: 0.7747 - categorical_accuracy: 0.7440 - val_loss: 0.9895 - val_categorical_accuracy: 0.

In [9]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 2,574,046
Trainable params: 2,574,046
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 137s 13ms/step - loss: 1.2159 - categorical_accuracy: 0.6058 - val_loss: 1.4586 - val_categorical_accuracy: 0.5250
Epoch 2/20
10628/10628 [==============================] - 137s 13ms/step - loss: 0.6712 - categorical_accuracy: 0.7743 - val_loss: 0.9205 - val_categorical_accuracy: 0.6951
E

In [10]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 2,665,438
Trainable params: 2,665,438
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 169s 16ms/step - loss: 1.3530 - categorical_accuracy: 0.5631 - val_loss: 1.4242 - val_categorical_accuracy: 0.4912
Epoch 2/20
10628/10628 [==============================] - 168s 16ms/step - loss: 0.9449 - categorical_accuracy: 0.6951 - val_loss: 1.3427 - val_categorical_accuracy: 0.4912
E

In [11]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_3 (GRU)                  (None, 512)               941568    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 3,243,998
Trainable params: 3,243,998
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 139s 13ms/step - loss: 1.2639 - categorical_accuracy: 0.6029 - val_loss: 1.1532 - val_categorical_accuracy: 0.6090
Epoch 2/20
10628/10628 [==============================] - 138s 13ms/step - loss: 0.6320 - categorical_accuracy: 0.7904 - val_loss: 0.9043 - val_categorical_accuracy: 0.7166
E

In [12]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1126 03:13:37.980021 140615897368384 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_4 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
Total params: 2,605,662
Trainable params: 2,605,662
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 159s 15ms/step - loss: 1.

In [13]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_5 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                650       
Total params: 2,588,574
Trainable params: 2,588,574
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 158s 15ms/step - loss: 1.

In [14]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_6 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_9 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                330       
Total params: 2,580,030
Trainable params: 2,580,030
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 160s 15ms/step - loss: 1.

In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 2,574,046
Trainable params: 2,574,046
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 157s 15ms/step - loss: 1.3335 - categorical_accuracy: 0.5629 - val_loss: 1.1271 - val_categorical_accuracy: 0.6308
Epoch 2/20
10628/10628 [==============================] - 156s 15ms/step - loss: 0.7811 - categorical_accuracy: 0.7412 - val_loss: 0.8792 - val_categorical_accuracy: 0.7094
E

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 100)          2297300   
_________________________________________________________________
gru_8 (GRU)                  (None, 500, 256)          274176    
_________________________________________________________________
gru_9 (GRU)                  (None, 64)                61632     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                650       
Total params: 2,633,758
Trainable params: 2,633,758
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 315s 30ms/step - loss: 1.3242 - categorical_accuracy: 0.5615 - val_loss: 1.3519 - val_categorical_accuracy: 0.5374
Epoch 2/20
10628/10628 [==================

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(GRU(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
gru_10 (GRU)                 (None, 500, 256)          274176    
_________________________________________________________________
gru_11 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 2,599,550
Trainable params: 2,599,550
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 321s 30ms/step - loss: 1.3041 - categorical_accuracy: 0.5751 - val_loss: 1.6338 - val_categorical_accuracy: 0.5371
Epoch 2/20
10628/10628 [==================

In [19]:
from keras.layers import Bidirectional, LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                5130      
Total params: 3,033,566
Trainable params: 3,033,566
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 407s 38ms/step - loss: 1.4202 - categorical_accuracy: 0.5433 - val_loss: 1.3721 - val_categorical_accuracy: 0.5713
Epoch 2/20
10628/10628 [==============================] - 429s 40ms/step - loss: 1.0343 - categorical_accuracy: 0.6628 - val_loss: 1.2187 - val_categorical_accuracy: 0.5920
E

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
gru_12 (GRU)                 (None, 512)               941568    
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                2570      
Total params: 3,372,766
Trainable params: 3,372,766
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 177s 17ms/step - loss: 1.

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
gru_13 (GRU)                 (None, 512)               941568    
_________________________________________________________________
dense_17 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                1290      
Total params: 3,305,822
Trainable params: 3,305,822
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 159s 15ms/step - loss: 1.

In [22]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
gru_14 (GRU)                 (None, 512)               941568    
_________________________________________________________________
dense_19 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                650       
Total params: 3,272,350
Trainable params: 3,272,350
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 165s 16ms/step - loss: 1.

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
gru_15 (GRU)                 (None, 512)               941568    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 3,243,998
Trainable params: 3,243,998
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 163s 15ms/step - loss: 1.3157 - categorical_accuracy: 0.5685 - val_loss: 1.3694 - val_categorical_accuracy: 0.5510
Epoch 2/20
10628/10628 [==============================] - 160s 15ms/step - loss: 0.7839 - categorical_accuracy: 0.7492 - val_loss: 0.8245 - val_categorical_accuracy: 0.7328
E

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
gru_16 (GRU)                 (None, 500, 512)          941568    
_________________________________________________________________
gru_17 (GRU)                 (None, 128)               246144    
_________________________________________________________________
dense_22 (Dense)             (None, 10)                1290      
Total params: 3,486,302
Trainable params: 3,486,302
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 317s 30ms/step - loss: 1.3168 - categorical_accuracy: 0.5646 - val_loss: 1.0359 - val_categorical_accuracy: 0.6609
Epoch 2/20
10628/10628 [==================

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
gru_18 (GRU)                 (None, 500, 512)          941568    
_________________________________________________________________
gru_19 (GRU)                 (None, 64)                110784    
_________________________________________________________________
dense_23 (Dense)             (None, 10)                650       
Total params: 3,350,302
Trainable params: 3,350,302
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 320s 30ms/step - loss: 1.3269 - categorical_accuracy: 0.5649 - val_loss: 1.2136 - val_categorical_accuracy: 0.6199
Epoch 2/20
10628/10628 [==================

In [26]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               1255424   
_________________________________________________________________
dense_24 (Dense)             (None, 10)                5130      
Total params: 3,557,854
Trainable params: 3,557,854
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 203s 19ms/step - loss: 1.4496 - categorical_accuracy: 0.5286 - val_loss: 1.3912 - val_categorical_accuracy: 0.5175
Epoch 2/20
10628/10628 [==============================] - 200s 19ms/step - loss: 1.0526 - categorical_accuracy: 0.6616 - val_loss: 1.2971 - val_categorical_accuracy: 0.5916
E

In [27]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 500, 100)          2297300   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              2510848   
_________________________________________________________________
dense_25 (Dense)             (None, 10)                10250     
Total params: 4,818,398
Trainable params: 4,818,398
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 390s 37ms/step - loss: 1.4653 - categorical_accuracy: 0.5219 - val_loss: 1.6000 - val_categorical_accuracy: 0.4840
Epoch 2/20
10628/10628 [==============================] - 386s 36ms/step - loss: 1.0491 - categorical_accuracy: 0.6608 - val_loss: 1.1936 - val_categorical_accuracy: 0.6048
E